In [2]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

app = JupyterDash(__name__)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# Create list of dictionaries for options in dropdown
dict_1 = {'label':'All sites', 'value':'ALL'}
list_of_dict = [dict_1]
for i in spacex_df['Launch Site'].unique():
    list_of_dict.append({'label':i, 'value':i})
    
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                            options=list_of_dict,
                                            value = 'ALL',
                                            placeholder = 'Select a Launch Site',
                                            searchable = True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                               min = 0, max = 10000, step = 1000,
                                               value = [min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                
                                html.Br(),
                                html.Div(id='percentage-output'),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    filtered_df = spacex_df.loc[spacex_df['Launch Site'] == entered_site]
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, 
                    values = 'class',
                    names = 'Launch Site',
                    title = 'Total Success Launches by site')
        return fig
    
    else:
        fig = px.pie(filtered_df,
                    names = 'class',
                    title = f'Total Success Launches for site {entered_site}')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    Input('site-dropdown','value'),
    Input('payload-slider','value'))

def get_scatter_chart(entered_site, mass_range):
    filtered_df = spacex_df.loc[spacex_df['Launch Site'] == entered_site]
    if entered_site == 'ALL':
        low = mass_range[0]
        high = mass_range[1]
        mask = (spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)
        spacex_dff = spacex_df[mask]
        fig = px.scatter(
            spacex_dff, x='Payload Mass (kg)', y='class',
            color='Booster Version Category')
        return fig
    else:
        low = mass_range[0]
        high = mass_range[1]
        mask = (spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)
        filtered_dff = filtered_df[mask]
        fig = px.scatter(
            filtered_dff, x='Payload Mass (kg)', y = 'class',
            color='Booster Version Category')
        return fig

@app.callback(
    Output('percentage-output', 'children'),
    Input('site-dropdown','value'),
    Input('payload-slider', 'value'))

def get_percentage(entered_site, mass_range):
    filtered_df = spacex_df.loc[spacex_df['Launch Site'] == entered_site]
    if entered_site == 'ALL':
        low = mass_range[0]
        high = mass_range[1]
        mask = (spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)
        spacex_dff = spacex_df[mask]
        success_pct = sum(spacex_dff['class'])/len(spacex_dff['class'])*100
        return f'Launch success rate for Payload Mass between {low}kg and {high}kg and for all sites:  {success_pct:.2f}%'
    else:
        low = mass_range[0]
        high = mass_range[1]
        mask = (filtered_df['Payload Mass (kg)'] >= low) & (filtered_df['Payload Mass (kg)'] <= high)
        filtered_dff = filtered_df[mask]
        success_pct = sum(filtered_dff['class'])/len(filtered_dff['class'])*100
        return f'Launch success rate for Payload Mass between {low}kg and {high}kg and for {entered_site} launch site:  {success_pct:.2f}%'

In [4]:
app.run_server()

Dash app running on http://127.0.0.1:8050/


1. KSC LC-39A has largest successful launches
2. 
3. 